In [ ]:
# # Install required libraries
# !pip install gdown -q
# !pip install transformers -q
# !pip install seaborn -q

# # Download dataset from Google Drive
# !gdown --id 1N740x0d6go1RDi_ULrGUy71pM76QGxQU -O dataset.zip

# # Unzip the downloaded dataset
# !unzip -o dataset.zip -d ./data

In [ ]:
# # Paths to the dataset directories
# og_arxiv_path = './data/og/arxiv'  # Original (ORIG) texts
# mg_arxiv_path = './data/bert-large-cased_parallel_mlm_prob_0.2/mg/arxiv'  # Machine-generated (SPUN) texts

og_arxiv_path = 'MachineParaphraseCorpus/og/arxiv'  # Original (ORIG) texts
mg_arxiv_path = 'MachineParaphraseCorpus/bert-large-cased_parallel_mlm_prob_0.2/mg/arxiv'  # Machine-generated (SPUN) texts

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# List of original and paraphrased files
original_files = [f for f in os.listdir(og_arxiv_path) if f.endswith('.txt')]
paraphrased_files = [f for f in os.listdir(mg_arxiv_path) if f.endswith('.txt')]

# Initialize data list
data = []

# Match and read files
for para_file in paraphrased_files:
    # Extract base filename to find corresponding original file
    base_filename = para_file.split('-SPUN')[0] + '.txt'  # Adjust the split as per your filenames
    original_file = base_filename

    if original_file in original_files:
        # Read original text
        with open(os.path.join(og_arxiv_path, original_file), 'r', encoding='utf-8') as f:
            original_text = f.read()

        # Read paraphrased text
        with open(os.path.join(mg_arxiv_path, para_file), 'r', encoding='utf-8') as f:
            paraphrased_text = f.read()

        # Append positive example
        data.append({'original': original_text, 'paraphrased': paraphrased_text, 'label': 1})


In [ ]:
# Match and create positive examples
for para_file in paraphrased_files:
    # Extract the prefix and number from the paraphrased file
    prefix = para_file.split('-SPUN')[0]  # Extract '0704.0097'
    number = para_file.split('-SPUN-')[-1].split('.')[0]  # Extract '3'

    # Construct the corresponding original filename
    original_file = f"{prefix}-ORIG-{number}.txt"

    # Check if the corresponding original file exists
    if original_file in original_files:
        # Read the original text
        with open(os.path.join(og_arxiv_path, original_file), 'r', encoding='utf-8') as f:
            original_text = f.read()

        # Read the paraphrased text
        with open(os.path.join(mg_arxiv_path, para_file), 'r', encoding='utf-8') as f:
            paraphrased_text = f.read()

        # Append the positive example
        data.append({'original': original_text, 'paraphrased': paraphrased_text, 'label': 1})

# Combine positive and negative samples
full_data = data + negative_data
df = pd.DataFrame(full_data)
print(f"Total dataset size: {len(df)}")
print(df['label'].value_counts())  # Check label distribution

# Prepare data for training
texts = list(zip(df['original'], df['paraphrased']))
labels = df['label']


In [ ]:
# Train-Test Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels)

# Load Pretrained Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize Text
train_encodings = tokenizer(
    [t[0] for t in train_texts], [t[1] for t in train_texts], truncation=True, padding=True)
val_encodings = tokenizer(
    [t[0] for t in val_texts], [t[1] for t in val_texts], truncation=True, padding=True)

# Custom Dataset Class
class ParaphraseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist()  # Ensure labels are in list format

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ParaphraseDataset(train_encodings, train_labels)
val_dataset = ParaphraseDataset(val_encodings, val_labels)

# # Training Arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy="epoch",
#     per_device_train_batch_size=8,  # Adjust based on available GPU memory
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     logging_dir='./logs',
#     logging_steps=10,
#     save_strategy='no'
# )

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,  # Adjust based on GPU memory
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='no'
)

# Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Save Model in PyTorch
def save_model_pytorch(model, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model_path = os.path.join(output_dir, "model.pth")
    print(f"Saving model to {model_path}...")
    torch.save(model.state_dict(), model_path)
    print("Model saved successfully!")

# Load Model in PyTorch
def load_model_pytorch(model_class, model_path, config):
    print(f"Loading model from {model_path}...")
    model = model_class.from_pretrained(None, config=config)
    model.load_state_dict(torch.load(model_path))
    print("Model loaded successfully!")
    return model


In [ ]:
# Train the Model
trainer.train()

In [ ]:
# Evaluate the Model
eval_results = trainer.evaluate()
print("\nEvaluation Results:")
for key, value in eval_results.items():
    if key.startswith("eval_"):
        print(f"{key}: {value:.4f}")

# Generate Predictions
predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
print("\nConfusion Matrix:")
print(cm)

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Original', 'Paraphrased'],
            yticklabels=['Original', 'Paraphrased'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=['Original', 'Paraphrased']))